## 自動微分
**PyTorch はそのテンソルに適用されるすべての操作を記録する。微分した値はgradに保存される**

In [6]:
import torch

t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)
t_un = 0.1 * t_u

In [2]:
def model(t_u, w, b):
    return t_u * w + b

def loss_fn(t_p, t_c):
    '''
    平均二乗誤差
    '''
    squared_diffs = (t_p - t_c) **2
    return squared_diffs.mean()

# パラメータを初期化: requires_grad=True(自動微分を有効化)
params = torch.tensor([1.0, 0], requires_grad=True)
# print(params.grad) None

# 勾配を計算
loss = loss_fn(model(t_u, *params), t_c)
loss.backward()
print(params.grad)

tensor([4517.2969,   82.6000])


### 勾配関数の累積

In [3]:
if params.grad is not None:
    params.grad.zero_()

In [4]:
def training_loop(n_epochs, learning_rate, params, t_u, t_c,
                  print_params=True):
    for epoch in range(1, n_epochs + 1):
        if params.grad is not None:
            params.grad.zero_()

        t_p = model(t_u, *params)  # <1>
        # 予測値と実測値の平均二乗誤差を計算
        loss = loss_fn(t_p, t_c)
        # 勾配を計算
        loss.backward()

        with torch.no_grad():
            # パラメータを更新
            params -= learning_rate * params.grad

        if epoch % 500 == 0:  # <3>
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return params

In [7]:
params = training_loop(
    n_epochs = 5000, 
    learning_rate = 1e-2, 
    params = torch.tensor([1.0, 0.0], requires_grad=True), 
    t_u = t_un, 
    t_c = t_c,)

Epoch 500, Loss 7.860115
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957698
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647
